In [1]:
import random

from utils.preprocess import load
from utils.markov_chain import MarkovChain

In [2]:
lines = load('texts/voina_i_mir.txt')
chain = MarkovChain(seqlen=6)

In [3]:
chain.train(lines)

In [4]:
prefix = "в голове "
evals = [(i, chain.inference(prefix=prefix)) for i in range(20)]

In [5]:
evals

[(0, 'в голове и.'),
 (1, 'в голове одному из кабинет.'),
 (2, 'в голове вся над русская войны.'),
 (3, 'в голове одно и только смеявшись от устройство и житейское.'),
 (4, 'в голове в ножны.'),
 (5, 'в голове ее.'),
 (6, 'в голове приходя с пасущимся выставил денисова особенно'),
 (7, 'в голове его в темно синий с краснея шеей.'),
 (8, 'в голове одной поезде.'),
 (9, 'в голове ясно слышишь.'),
 (10, 'в голове петербурге.'),
 (11, 'в голове его в.'),
 (12, 'в голове его всю ночь чтобы империю и французом мог так я'),
 (13, 'в голове от.'),
 (14, 'в голове пети.'),
 (15, 'в голове одной босой.'),
 (16, 'в голове пети.'),
 (17, 'в голове становилась себе ее по горючьми слезы и тяжестями'),
 (18, 'в голове одной мысли прежде было и не может быть осрамить.'),
 (19, 'в голове его горя грустного министр иностранившегося')]

In [6]:
from utils.preprocess import alphabet
import numpy as np

In [7]:
char2index, index2char = alphabet(lines)
text = ' '.join(lines)
symbols_len = len(char2index)

In [8]:
len(lines)

63239

In [9]:
SEQ_LEN = 30
WINDOW_STEP = 1

In [10]:
inputs = []
targets = []

for i in range(0, len(text) - SEQ_LEN, WINDOW_STEP):
    inputs.append(text[i: i + SEQ_LEN])
    targets.append(text[i + SEQ_LEN])

In [11]:
def encode_line(seq):
    x = np.zeros((len(seq), symbols_len))
    # encode sequence
    for j, char in enumerate(seq):
        x[j, char2index[char]] = 1
    return list(x)

def decode_line(seq):
    result = []
    for j in seq:
        char_index = np.argmax(j)
        char = index2char[char_index]
        result.append(char)
    result = ''.join(result)
    return result

In [12]:
decode_line(encode_line('кес'))

'кес'

In [13]:
X = []
Y = []

for i, seq in enumerate(inputs):
    x = encode_line(seq)
    y = encode_line(targets[i])[0]
    X.append(x)
    Y.append(y)

In [14]:
all_data = list(zip(X, Y))[:400000]
# all_data = random.choices(all_data, k=400000)
X, Y = list(zip(*all_data))
X, Y = np.array(X), np.array(Y)

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [16]:
X.shape, Y.shape

((400000, 30, 34), (400000, 34))

In [17]:
X_train.shape, len(y_train)

((320000, 30, 34), 320000)

In [18]:
import tensorflow as tf
lstm = tf.keras.models.Sequential()
lstm.add(
    tf.keras.layers.LSTM(
        128,
        input_shape=(SEQ_LEN, symbols_len),
        dropout=0.4,
        recurrent_dropout=0.4
    )
)
lstm.add(tf.keras.layers.Dense(symbols_len, activation='softmax'))
lstm.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
     

2023-09-17 20:58:06.688225: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
lstm.fit(X_train, y_train, 
        epochs=20, 
        validation_data=(X_test, y_test),
        use_multiprocessing=True,
        workers=-1
)

Epoch 1/20
10000/10000 [==============================] - 209s 21ms/step - loss: 2.5211 - accuracy: 0.2662 - val_loss: 2.1956 - val_accuracy: 0.3406
Epoch 2/20
10000/10000 [==============================] - 179s 18ms/step - loss: 2.2677 - accuracy: 0.3219 - val_loss: 2.0585 - val_accuracy: 0.3737
Epoch 3/20
10000/10000 [==============================] - 193s 19ms/step - loss: 2.1807 - accuracy: 0.3429 - val_loss: 1.9816 - val_accuracy: 0.3923
Epoch 4/20
 6927/10000 [===================>..........] - ETA: 58s - loss: 2.1352 - accuracy: 0.3536

In [ ]:
def sample(predictions):
    predictions = np.log(predictions)
    exp_predictions = np.exp(predictions)
    predictions = exp_predictions / np.sum(exp_predictions)
    probabilities = np.random.multinomial(1, predictions, 1)
    return np.argmax(probabilities)

def lstm_predict(model, string, max_len=None):
    print(string, end="")
    result = list(string)
    string = encode_line(string)
    while True:
        prediction = model.predict(np.array([string]))
        # ind = sample(prediction[0])
        # ind = np.argmax(prediction[0])
        # char = index2char[ind]
        char = random.choices(index2char, weights=prediction[0])[0]
        string.append(encode_line(char)[0])
        result.append(char)
        if (char == '.' and not max_len) or (max_len and len(result) > max_len):
            break
        
    return ''.join(result)

In [ ]:
example = "вместо тысящи слов о дружбе н "
lstm_predict(lstm, example)

In [ ]:
prefixes = [
    "жить во ",
    "по лицу ",
    "смысл жизни в том ",
    "фактически ",
    "спросил я "
]

In [ ]:
result = []
for n, pref in enumerate(prefixes):
    result.append((
        n,
        ("markov: " + chain.inference(pref)),
        ("lstm: " + lstm_predict(lstm, pref))
    ))
    # print("markov:", chain.inference(pref, max_length=80))
    # print("lstm:", lstm_predict(lstm, pref))

In [ ]:
for t in result:
    print(t[0])
    print(t[1])
    print(t[2])

In [ ]:
lstm_predict(lstm, 'мне нравится как говорят киты ', max_len=300)